In [170]:
# This block is to install ase, tk for the GUI, and ipywidgets-jsonschema
import sys

print(sys.executable)
# !{sys.executable} -m pip install ase
# !{sys.executable} -m pip install tk
import tkinter

print(tkinter.Tcl().eval("info patchlevel"))
# !{sys.executable} -m pip install ipywidgets-jsonschema


/Users/mat3ra/.pyenv/versions/3.11.4/bin/python
8.6.13


In [171]:
# data that comes from JS, here are some examples
poscars = {
    "Ni": """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
direct
   0.000000000    0.000000000    0.000000000 T T T Ni
""",
    "Cu": """Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
    "Au": """Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
    "SiC": """Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
""",
    "Graphene": """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
}


In [310]:
import io
from ase.build import surface, make_supercell
from ase.io import read, write
from ase import Atoms
import numpy as np
from ase.visualize import view

# Define a function to read POSCAR content and return an Atoms object
def poscar_to_atoms(poscar_content):
    poscar_io = io.StringIO(poscar_content)
    atoms = read(poscar_io, format="vasp")
    return atoms

# Define a function to create a surface and supercell
def create_surface_and_supercell(atoms, miller, number_of_layers, vacuum, superlattice_matrix):
    miller_indices = (miller["h"], miller["k"], miller["l"])
    surface_atoms = surface(atoms, miller_indices, number_of_layers, vacuum)
    supercell_atoms = make_supercell(surface_atoms, superlattice_matrix)
    return supercell_atoms

# Define a function to calculate strain on the layer
def calculate_strain_matrix(original_layer_cell, scaled_layer_cell):
    # Calculate the original and scaled norms
    cell1 = np.array(original_layer_cell)[0:2,0:2]
    cell2 = np.array(scaled_layer_cell)[0:2,0:2]
    transformation_matrix = np.linalg.inv(cell1) @ cell2
    difference_matrix = transformation_matrix - np.eye(2)
    return difference_matrix

# Define the main function to create the interface
def create_interface(substrate_poscar, layer_poscar, substrate_surface_params, layer_surface_params, z_offset, force_wrap):
    # Convert POSCAR content to Atoms objects
    substrate_atoms = poscar_to_atoms(substrate_poscar)
    layer_atoms = poscar_to_atoms(layer_poscar)
    
    # Create surface and supercell for substrate
    substrate_supercell = create_surface_and_supercell(substrate_atoms, **substrate_surface_params)

    # Create surface and supercell for layer
    layer_supercell = create_surface_and_supercell(layer_atoms, **layer_surface_params)

    # Calculate strain on the layer
    m = calculate_strain_matrix(substrate_supercell.get_cell(), layer_supercell.get_cell())
    m_percent = m * 100 
    percent_str = np.array2string(m_percent, formatter={'float': '{:0.2f}%'.format})
    print("Strain matrix as percentages:\n", percent_str)


    # Adjust Z position based on the Z offset
    z_max_substrate = max(substrate_supercell.positions[:, 2])
    layer_supercell.positions[:, 2] += z_max_substrate + z_offset
    
    # Combine substrate and layer into one Atoms object
    interface = substrate_supercell + layer_supercell
    if force_wrap: interface.wrap()
    return interface




In [320]:

# Example usage
substrate_poscar = poscars["Au"]
layer_poscar = poscars["Graphene"]
substrate_surface_params = {
    "miller": {"h": 1, "k": 1, "l": 0},
    "number_of_layers": 3,
    "vacuum": 3,
    "superlattice_matrix": [[2, 0, 0], [0, 2, 0], [0, 0, 1]],
}
layer_surface_params = {
    "miller": {"h": 0, "k": 0, "l": 1},
    "number_of_layers": 1,
    "vacuum": 3,
    "superlattice_matrix": [[4, -1, 0], [2, 4, 0], [0, 0, 1]],
}
z_offset = 0.0

interface_atoms = create_interface(substrate_poscar, layer_poscar, substrate_surface_params,layer_surface_params, z_offset, force_wrap=False)

view(interface_atoms*(2,2,1))




Strain matrix as percentages:
 [[-5.89% -18.11%]
 [0.00% 2.45%]]


<Popen: returncode: None args: ['/Users/mat3ra/.pyenv/versions/3.11.4/bin/py...>

In [321]:
# Gr/Au(110) -- large mismatch
substrate_poscar = poscars["Au"]
layer_poscar = poscars["Graphene"]
substrate_surface_params = {
    "miller": {"h": 1, "k": 1, "l": 0},
    "number_of_layers": 3,
    "vacuum": 3,
    "superlattice_matrix": [[2, 0, 0], [0, 2, 0], [0, 0, 1]],
}
layer_surface_params = {
    "miller": {"h": 0, "k": 0, "l": 1},
    "number_of_layers": 1,
    "vacuum": 3,
    "superlattice_matrix": [[4, 0, 0], [0, 4, 0], [0, 0, 1]],
}
z_offset = 0.0

interface_atoms = create_interface(substrate_poscar, layer_poscar, substrate_surface_params,layer_surface_params, z_offset, force_wrap=False)

view(interface_atoms*(2,2,1))



Strain matrix as percentages:
 [[-16.35% 0.00%]
 [-59.15% 2.45%]]


<Popen: returncode: None args: ['/Users/mat3ra/.pyenv/versions/3.11.4/bin/py...>

In [322]:
# Gr/Au(110) -- small mismatch
substrate_poscar = poscars["Au"]
layer_poscar = poscars["Graphene"]
substrate_surface_params = {
    "miller": {"h": 1, "k": 1, "l": 0},
    "number_of_layers": 3,
    "vacuum": 3,
    "superlattice_matrix": [[2, 0, 0], [0, 2, 0], [0, 0, 1]],
}
layer_surface_params = {
    "miller": {"h": 0, "k": 0, "l": 1},
    "number_of_layers": 1,
    "vacuum": 3,
    "superlattice_matrix": [[4, -1, 0], [2, 4, 0], [0, 0, 1]],
}
z_offset = 0.0

interface_atoms = create_interface(substrate_poscar, layer_poscar, substrate_surface_params,layer_surface_params, z_offset, force_wrap=False)

view(interface_atoms*(2,2,1))



Strain matrix as percentages:
 [[-5.89% -18.11%]
 [0.00% 2.45%]]


<Popen: returncode: None args: ['/Users/mat3ra/.pyenv/versions/3.11.4/bin/py...>

In [275]:
substrate_poscar = poscars["SiC"]
layer_poscar = poscars["Graphene"]
surface_params = {
    "miller": {"h": 1, "k": 1, "l": 1},
    "number_of_layers": 3,
    "vacuum": 10,
}
substrate_atoms = poscar_to_atoms(substrate_poscar)  # Use the actual POSCAR content
surface_atoms = surface(substrate_atoms, (surface_params["miller"]["h"], surface_params["miller"]["k"], surface_params["miller"]["l"]), surface_params["number_of_layers"], surface_params["vacuum"])
layer_atoms = poscar_to_atoms(layer_poscar)     


# Brute force search for the best superlattice matrices
min_val, max_val = 0, 2  
max_increase_percentage = 100
best_matrices = search_best_matrices(surface_atoms, layer_atoms, min_val, max_val, max_increase_percentage)


# Print the best result
if best_matrices:
    best_result = best_matrices[0]
    print("Best superlattice matrices with the smallest total strain:")
    print("Strain along a:", best_result["strain_a"])
    print("Strain along b:", best_result["strain_b"])
    print("Total strain:", best_result["total_strain"])
    print("Substrate superlattice matrix:", best_result["slab_matrix"])
    print("Layer superlattice matrix:", best_result["layer_matrix"])
else:
    print("No valid superlattice matrices found within the given range.")

# Visualize the best result
if best_matrices:
    superlattice_matrices = {
        "slab": best_result["slab_matrix"],
        "layer": best_result["layer_matrix"],
    }
    z_offset = 3.0  # Example Z offset
    
    interface_atoms = create_interface(substrate_poscar, layer_poscar, surface_params, superlattice_matrices, z_offset)
    view(interface_atoms*(3,3,1))
    print(interface_atoms)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()